In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
import tensorflow as tf
import keras


mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

c:\Users\Emeline_PRO\Documents\Modules\Module_ACV\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# New version 

In [2]:
def get_landmarks(image_rgb, pose_model): 
    '''
    Récupère les landmarks d'une image donnée en RGB via un modèle déjà chargé.
    Return: un numpy array (33,4) ou None si rien n'est trouvé.
    '''

    results = pose_model.process(image_rgb)
    
    if results.pose_landmarks:

        pose_np = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
        return pose_np

    return None

def mmss(seconds: float) -> str:
    s = max(0, int(seconds))
    return f"{s//60:02d}:{s%60:02d}"

def draw_rounded_rect(img, pt1, pt2, radius=22, color=(255, 200, 230), thickness=-1):
    x1, y1 = pt1
    x2, y2 = pt2
    radius = int(max(0, min(radius, abs(x2-x1)//2, abs(y2-y1)//2)))
    if thickness < 0:
        cv2.rectangle(img, (x1+radius, y1), (x2-radius, y2), color, -1)
        cv2.rectangle(img, (x1, y1+radius), (x2, y2-radius), color, -1)
        cv2.circle(img, (x1+radius, y1+radius), radius, color, -1)
        cv2.circle(img, (x2-radius, y1+radius), radius, color, -1)
        cv2.circle(img, (x1+radius, y2-radius), radius, color, -1)
        cv2.circle(img, (x2-radius, y2-radius), radius, color, -1)
    else:
        cv2.rectangle(img, pt1, pt2, color, thickness)

def overlay_alpha(dst, overlay, alpha=0.75):
    cv2.addWeighted(overlay, alpha, dst, 1-alpha, 0, dst)

def put_fit_text(img, text, org, max_width, font=cv2.FONT_HERSHEY_SIMPLEX, base_scale=1.0, thickness=2, color=(60,30,60)):
    (tw, th), _ = cv2.getTextSize(text, font, base_scale, thickness)
    scale = base_scale if tw <= max_width else base_scale * (max_width / tw)
    cv2.putText(img, text, org, font, scale, color, thickness, cv2.LINE_AA)

def draw_session_card(frame, session, idx, paused=False, phase="HOLD"):
    x, y = 15, 15
    box_w, box_h = 220, 70
    draw_rounded_rect(frame, (x, y), (x+box_w, y+box_h), radius=12, color=(255, 245, 250), thickness=-1)
    
    cv2.putText(frame, "YOGA", (x+12, y+22), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (60,30,60), 1)
    cv2.putText(frame, f"{idx+1}/{len(session)}", (x+box_w-45, y+22), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (60,30,60), 1)
    
    posture_name = session[idx]["name"].upper()
    put_fit_text(frame, posture_name, (x+12, y+55), max_width=box_w-25, base_scale=0.8)
    
    status_text = "PAUSED" if paused else f"STATUS: {phase}"
    cv2.putText(frame, status_text, (x+15, y+85), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 0, 100), 1)

def draw_progress_bar_bottom(frame, progress, right_text=""):
    H, W = frame.shape[:2]
    margin, bar_h = 25, 20
    x, y, w = margin, H - 45, W - 2 * margin
    cv2.rectangle(frame, (x, y), (x+w, y+bar_h), (240, 230, 240), -1) # BG
    cv2.rectangle(frame, (x, y), (x+int(w*progress), y+bar_h), (80, 0, 80), -1) # Fill
    cv2.rectangle(frame, (x, y), (x+w, y+bar_h), (60, 30, 60), 1) # Border
    if right_text:
        cv2.putText(frame, right_text, (x+3, y+15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (220, 0, 220), 1)

def draw_controls(image, is_paused):
    h, w, _ = image.shape
    text = "P:PAUSE  N:NEXT  Q:QUIT"
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.4
    thickness = 1
    
    (text_w, text_h), _ = cv2.getTextSize(text, font, scale, thickness)
    
    margin = 15
    rect_x1, rect_y1 = w - text_w - 20, margin
    rect_x2, rect_y2 = w - margin, margin + text_h + 12
    
    cv2.rectangle(image, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), -1)
    cv2.putText(image, text, (rect_x1 + 5, rect_y2 - 6), font, scale, (255, 255, 255), thickness)

    if is_paused:
        overlay = image.copy()
        cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.3, image, 0.7, 0, image)
        cv2.putText(image, "PAUSED", (w//2 - 60, h//2), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255), 3)

def draw_end_screen(image):
    h, w, _ = image.shape
    overlay = image.copy()
    cv2.rectangle(overlay, (0, 0), (w, h), (255, 240, 250), -1)
    cv2.addWeighted(overlay, 0.8, image, 0.2, 0, image)
    
    cv2.putText(image, "END !", (w//2 - 100, h//2 - 60), cv2.FONT_HERSHEY_DUPLEX, 2.5, (60, 30, 60), 5)
    cv2.putText(image, "FINISHED ! ", (w//2 - 150, h//2 + 20), cv2.FONT_HERSHEY_DUPLEX, 1.5, (60, 30, 60), 3)
    cv2.putText(image, "BRAVO !", (w//2 - 110, h//2 + 90), cv2.FONT_HERSHEY_DUPLEX, 1.2, (100, 0, 100), 2)
    cv2.putText(image, "Put 'Q' to quit", (w//2 - 115, h - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (50, 50, 50), 1)

In [3]:
model = tf.keras.models.load_model('yoga_pose_model.keras')
model.weights

[<Variable path=dense_21/kernel, shape=(132, 254), dtype=float32, value=[[-0.04493148 -0.12349138 -0.04918426 ... -0.02998154 -0.11770038
    0.12310738]
  [ 0.11831767 -0.02272718 -0.10223728 ... -0.0368409  -0.05330607
    0.0817641 ]
  [ 0.0326119  -0.09212064 -0.01958724 ...  0.09605832 -0.08689427
   -0.0305092 ]
  ...
  [-0.04350108  0.03959004  0.02002547 ...  0.07562789 -0.04124256
   -0.09257954]
  [ 0.03467342 -0.05395248 -0.08595873 ...  0.01378863  0.01250207
    0.01877752]
  [ 0.01150734 -0.02246944 -0.09911901 ... -0.09200414 -0.04097784
    0.0292645 ]]>,
 <Variable path=dense_21/bias, shape=(254,), dtype=float32, value=[ 1.63956010e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -5.37146907e-03  3.13364831e-03  0.00000000e+00  8.53680819e-03
   1.39132589e-02 -1.31112747e-02 -3.88627732e-03 -1.57265493e-03
   9.53475572e-03 -5.30584063e-03  5.60042262e-03 -5.98864397e-03
   0.00000000e+00  3.64685198e-03 -5.32775186e-03 -5.30599430e-03
   7.02245301e-03  1.002479

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

# 0= cobra  1 = Tree 2 = Dog
labels = ['Cobra', 'Tree', 'Dog']
model = tf.keras.models.load_model('yoga_pose_model.keras')

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

session = [
    {"name": "Cobra", "duration": 15},
    {"name": "Tree", "duration": 20},
    {"name": "Dog", "duration": 10}
]

cap = cv2.VideoCapture(0)
cam_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cam_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
zoom = 1.2
cv2.namedWindow("Yoga postures", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Yoga postures", int(cam_w*zoom), int(cam_h*zoom))

idx = 0
paused = False
start_time = None
elapsed_time = 0
is_preparing = True
is_finished = False 
prep_duration = 5 
prep_start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_model:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_model.process(image_rgb)
        landmarks_np = get_landmarks(image_rgb, pose_model)
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        phase = "WAITING..."
        progress = 0

        if is_finished:
            draw_end_screen(image_bgr)
            progress = 1 

        elif is_preparing:
            elapsed_prep = time.time() - prep_start_time
            countdown = int(prep_duration - elapsed_prep)
            
            cv2.putText(image_bgr, f"GOT READY : {session[idx]['name']}", (cam_w//2-180, cam_h//2-40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(image_bgr, str(countdown if countdown > 0 else "GO!"), (cam_w//2-30, cam_h//2+60), 
                        cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 0), 4)
            
            if elapsed_prep >= prep_duration:
                is_preparing = False
                start_time = None 
                elapsed_time = 0

        else:

            is_pose_correct = False

            if not paused:

                # Modele prediction posture 
                if landmarks_np is not None:
                    input_data = landmarks_np.flatten().reshape(1, -1) #format (33,4) --> (1, 132)
                    
                    prediction = model.predict(input_data, verbose=0)
                    predicted_idx = np.argmax(prediction)
                    confidence = prediction[0][predicted_idx]
                    
                    current_target_name = session[idx]["name"]
                    predicted_name = labels[predicted_idx]
                    
                if predicted_name == current_target_name and confidence > 0.9:
                    is_pose_correct = True

                #is_pose_correct = False
                #is_pose_correct = landmarks_np is not None # to test the model 
                
                if is_pose_correct:
                    phase = "HOLDING"

                    if start_time is None: 
                        start_time = time.time()
                    
                    elapsed_time = time.time() - start_time
                    target_duration = session[idx]["duration"]
                    progress = np.clip(elapsed_time / target_duration, 0, 1)

                    if elapsed_time >= target_duration:
                        if idx < len(session) - 1:
                            idx += 1
                            is_preparing = True
                            prep_start_time = time.time()
                            start_time = None
                            elapsed_time = 0
                        else:
                            is_finished = True 
                else:
                    start_time = None

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        draw_session_card(image_bgr, session, idx, paused=paused, phase=phase)
        remaining_time = max(0, session[idx]["duration"] - elapsed_time)
        draw_progress_bar_bottom(image_bgr, progress, right_text=mmss(remaining_time))
        draw_controls(image_bgr, paused)
        cv2.imshow('Yoga postures', image_bgr)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'): break
        elif key == ord('p'): paused = not paused
        elif key == ord('n'):
            if idx < len(session) - 1:
                idx += 1
                is_preparing = True
                prep_start_time = time.time()
                start_time = None
                elapsed_time = 0
            else:
                is_finished = True

    cap.release()
    cv2.destroyAllWindows()

In [5]:
prediction = model.predict(input_data, verbose=0)
predicted_idx = np.argmax(prediction)
